## Tracking Using CSRT

In [ ]:
import cv2
import numpy as np
import math
import os
import csv
import json
from statistics import median

# Utilities for resizing and point scaling

def resize_to_fit(frame, max_width=1280, max_height=720):
    h, w = frame.shape[:2]
    scale = min(max_width / w, max_height / h, 1.0)
    new_size = (int(w * scale), int(h * scale))
    return cv2.resize(frame, new_size), scale

def scale_point(p, scale):
    return (int(p[0] / scale), int(p[1] / scale))

# Mouse-based point selection
def select_point(frame, title="Select Pivot"):
    selected = {'pt': None}
    def callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            selected['pt'] = (x, y)
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, callback)
    while True:
        temp = frame.copy()
        if selected['pt']:
            cv2.circle(temp, selected['pt'], 5, (0,255,0), -1)
        cv2.imshow(title, temp)
        key = cv2.waitKey(1) & 0xFF
        # Enter to confirm, Esc to cancel
        if key == 13 and selected['pt']:
            break
        elif key == 27:
            cv2.destroyWindow(title)
            return None
    cv2.destroyWindow(title)
    return selected['pt']

# Mouse-based circular ROI selection
def manual_circle_selection(frame, title="Select Circle ROI"):
    state = {'center': None, 'radius': 0, 'drawing': False}
    def callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            state['center'] = (x, y)
            state['drawing'] = True
        elif event == cv2.EVENT_MOUSEMOVE and state['drawing']:
            state['radius'] = int(math.hypot(x - state['center'][0], y - state['center'][1]))
        elif event == cv2.EVENT_LBUTTONUP:
            state['drawing'] = False
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, callback)
    while True:
        temp = frame.copy()
        if state['center'] and state['radius'] > 0:
            cv2.circle(temp, state['center'], state['radius'], (0,255,0), 2)
        cv2.imshow(title, temp)
        key = cv2.waitKey(1) & 0xFF
        if key == 13 and state['center'] and state['radius'] > 0:
            break
        elif key == 27:
            cv2.destroyWindow(title)
            return None, None
    cv2.destroyWindow(title)
    return state['center'], state['radius']

# Main tracking function using CSRT

def track_objects(video_path, output_directory_path, frame_skipping=10):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video")
        return

    ret, first = cap.read()
    if not ret:
        print("Error reading first frame")
        return

    # Resize for selection UI
    resized, scale = resize_to_fit(first)

    # Select two pivots (for metadata)
    pt1_disp = select_point(resized, "Select Pivot 1")
    pt2_disp = select_point(resized, "Select Pivot 2")
    if not pt1_disp or not pt2_disp:
        print("Pivot selection cancelled")
        return
    pivot1 = scale_point(pt1_disp, scale)
    pivot2 = scale_point(pt2_disp, scale)

    # Select two circular ROIs around objects
    c1_disp, r1_disp = manual_circle_selection(resized, "Select Object 1 ROI")
    c2_disp, r2_disp = manual_circle_selection(resized, "Select Object 2 ROI")
    if not c1_disp or not c2_disp:
        print("Circle selection cancelled")
        return
    center1 = scale_point(c1_disp, scale)
    center2 = scale_point(c2_disp, scale)
    rad1 = int(r1_disp / scale)
    rad2 = int(r2_disp / scale)

    # Convert circles to bounding boxes (x, y, w, h)
    bbox1 = (center1[0] - rad1, center1[1] - rad1, rad1 * 2, rad1 * 2)
    bbox2 = (center2[0] - rad2, center2[1] - rad2, rad2 * 2, rad2 * 2)

    # Prepare output files
    os.makedirs(output_directory_path, exist_ok=True)
    csv_path = os.path.join(output_directory_path, "coordinates.csv")
    json_path = os.path.join(output_directory_path, "metadata.json")

    # Write CSV header
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['frame', 'obj1_x', 'obj1_y', 'obj2_x', 'obj2_y'])

    # Initialize CSRT trackers
    tracker1 = cv2.TrackerCSRT_create()
    tracker2 = cv2.TrackerCSRT_create()
    tracker1.init(first, bbox1)
    tracker2.init(first, bbox2)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for frame_idx in range(0, total_frames, frame_skipping):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break

        # Update tracking
        ok1, b1 = tracker1.update(frame)
        ok2, b2 = tracker2.update(frame)

        cx1 = cy1 = cx2 = cy2 = ''
        vis = frame.copy()

        if ok1:
            x1, y1, w1, h1 = [int(v) for v in b1]
            cx1 = x1 + w1//2
            cy1 = y1 + h1//2
            cv2.rectangle(vis, (x1,y1), (x1+w1, y1+h1), (255,0,0), 2)
            cv2.putText(vis, 'Obj 1', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
        if ok2:
            x2, y2, w2, h2 = [int(v) for v in b2]
            cx2 = x2 + w2//2
            cy2 = y2 + h2//2
            cv2.rectangle(vis, (x2,y2), (x2+w2, y2+h2), (0,255,0), 2)
            cv2.putText(vis, 'Obj 2', (x2, y2-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        # Draw pivots
        cv2.circle(vis, pivot1, 5, (0,0,255), -1)
        cv2.circle(vis, pivot2, 5, (0,0,255), -1)
        cv2.putText(vis, f"Frame: {frame_idx}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

        # Save frame data
        with open(csv_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([frame_idx, cx1, cy1, cx2, cy2])

        # Display
        disp, _ = resize_to_fit(vis)
        cv2.imshow('Tracking CSRT', disp)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Post-process: remove outliers
    rows = []
    with open(csv_path, 'r') as f:
        reader = list(csv.reader(f))
        header, rows = reader[0], reader[1:]

    clean = []
    distances = []
    for i in range(1, len(rows)):
        prev = rows[i-1]
        curr = rows[i]
        try:
            p1 = list(map(int, prev[1:3]))
            p2 = list(map(int, curr[1:3]))
            q1 = list(map(int, prev[3:5]))
            q2 = list(map(int, curr[3:5]))
        except ValueError:
            continue
        d = math.hypot(p2[0]-p1[0], p2[1]-p1[1]) + math.hypot(q2[0]-q1[0], q2[1]-q1[1])
        distances.append((i, d))

    med = median([d for _, d in distances]) if distances else 0
    thresh = 3 * med
    outliers = {i for i, d in distances if d > thresh}

    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for idx, row in enumerate(rows):
            if idx in outliers: continue
            if all(str(val).strip() for val in row[1:5]):
                writer.writerow(row)

    # Save metadata
    meta = {
        'obj1_pivot': pivot1,
        'obj2_pivot': pivot2,
        'initial_bbox1': bbox1,
        'initial_bbox2': bbox2
    }
    with open(json_path, 'w') as f:
        json.dump(meta, f, indent=4)

    print("Tracking complete. CSV and metadata saved.")


track_objects(
video_path=R"C:\Users\avsha\Documents\GitHub\Ants\tracked_penduli\vid6.mp4",
output_directory_path=R"C:\Users\avsha\Documents\GitHub\Ants\temp",
frame_skipping=1
)



Tracking complete. CSV and metadata saved.


### New Vesrion: Tracking Using Hugh Circles and Physical Constrains 

In [ ]:
# Circle detection with visualization, no ant logic, supports frame skipping
import cv2
import numpy as np
import math
import os
import csv
import json
from statistics import median

# Utilities (same as original)
def resize_to_fit(frame, max_width=1280, max_height=720):
    h, w = frame.shape[:2]
    scale = min(max_width / w, max_height / h, 1.0)
    new_size = (int(w * scale), int(h * scale))
    return cv2.resize(frame, new_size), scale

def scale_point(p, scale):
    return (int(p[0] / scale), int(p[1] / scale))

def select_point(frame, title="Select point"):
    selected = {'pt': None}
    def callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            selected['pt'] = (x, y)
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, callback)
    while True:
        temp = frame.copy()
        if selected['pt'] is not None:
            cv2.circle(temp, selected['pt'], 5, (0, 255, 0), -1)
        cv2.imshow(title, temp)
        key = cv2.waitKey(1) & 0xFF
        if key == 13 and selected['pt']: break
        elif key == 27: return None
    cv2.destroyWindow(title)
    return selected['pt']

def manual_circle_selection(frame, title="Select circle"):
    state = {'center': None, 'radius': 0, 'drawing': False}
    def callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            state['center'] = (x, y)
            state['drawing'] = True
        elif event == cv2.EVENT_MOUSEMOVE and state['drawing']:
            state['radius'] = int(math.hypot(x - state['center'][0], y - state['center'][1]))
        elif event == cv2.EVENT_LBUTTONUP:
            state['drawing'] = False
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, callback)
    while True:
        temp = frame.copy()
        if state['center']:
            if state['radius'] > 0:
                cv2.circle(temp, state['center'], state['radius'], (0, 255, 0), 2)
        cv2.imshow(title, temp)
        key = cv2.waitKey(1) & 0xFF
        if key == 13 and state['center'] and state['radius'] > 0: break
        elif key == 27: return None, None
    cv2.destroyWindow(title)
    return state['center'], state['radius']

def track_objects(video_path, output_directory_path, frame_skipping=10):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video")
        return

    ret, first = cap.read()
    if not ret:
        print("Error reading first frame")
        return

    resized, scale = resize_to_fit(first)
    pivot1 = scale_point(select_point(resized, "Select Pivot 1"), scale)
    center1_disp, radius1_disp = manual_circle_selection(resized, "Select Circle 1")
    center1 = scale_point(center1_disp, scale)

    pivot2 = scale_point(select_point(resized, "Select Pivot 2"), scale)
    center2_disp, radius2_disp = manual_circle_selection(resized, "Select Circle 2")
    center2 = scale_point(center2_disp, scale)

    dist1 = int(math.hypot(center1[0] - pivot1[0], center1[1] - pivot1[1]))
    rad1 = int(radius1_disp / scale)
    dist2 = int(math.hypot(center2[0] - pivot2[0], center2[1] - pivot2[1]))
    rad2 = int(radius2_disp / scale)

    hmin = max(1, min(rad1, rad2) - 15)
    hmax = max(rad1, rad2) + 15

    # Output paths
    csv_path = os.path.join(output_directory_path, f"coordinates.csv")
    json_path = os.path.join(output_directory_path, f"metadata.json")

    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['frame', 'obj1_x', 'obj1_y', 'obj2_x', 'obj2_y'])

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sm1, sm2 = None, None
        for i in range(0, total_frames, frame_skipping):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret: break

            vis = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_blur = cv2.GaussianBlur(gray, (7, 7), 0)
            circles = cv2.HoughCircles(gray_blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                                       param1=100, param2=30,
                                       minRadius=hmin, maxRadius=hmax)
            best1 = best2 = None
            if circles is not None:
                for x, y, r in np.uint16(np.around(circles[0])):
                    d1 = abs(math.hypot(x - pivot1[0], y - pivot1[1]) - dist1)
                    d2 = abs(math.hypot(x - pivot2[0], y - pivot2[1]) - dist2)
                    if d1 < 20 and abs(r - rad1) < 15: best1 = (x, y)
                    if d2 < 20 and abs(r - rad2) < 15: best2 = (x, y)

            writer.writerow([i, *(best1 if best1 else ('', '')), *(best2 if best2 else ('', ''))])

            # Visualization
            for (pivot, center, color) in [(pivot1, best1, (255, 255, 0)), (pivot2, best2, (255, 0, 255))]:
                cv2.circle(vis, pivot, 5, (0, 0, 255), -1)
                cv2.circle(vis, pivot, dist1 if pivot == pivot1 else dist2, (0, 255, 0), 1)
                if center:
                    cv2.circle(vis, center, rad1 if pivot == pivot1 else rad2, color, 2)
                    cv2.line(vis, pivot, center, color, 1)

            cv2.putText(vis, f"Frame: {i}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            disp, _ = resize_to_fit(vis)
            cv2.imshow("Tracking", disp)
            if cv2.waitKey(1) & 0xFF == ord('q'): break

        cap.release()
    cv2.destroyAllWindows()

    # Post-process CSV to remove outliers and clean up
    with open(csv_path, 'r') as f:
        reader = list(csv.reader(f))
        header, rows = reader[0], reader[1:]

    clean_rows = []
    dists = []
    for i in range(1, len(rows)):
        prev = rows[i - 1]
        curr = rows[i]

        try:
            prev_coords = list(map(int, prev[1:5]))
            curr_coords = list(map(int, curr[1:5]))
        except ValueError:
            continue  # Skip incomplete or malformed rows

        dx1 = curr_coords[0] - prev_coords[0]
        dy1 = curr_coords[1] - prev_coords[1]
        dx2 = curr_coords[2] - prev_coords[2]
        dy2 = curr_coords[3] - prev_coords[3]

        dist = math.hypot(dx1, dy1) + math.hypot(dx2, dy2)
        dists.append((i, dist))  # Store index and distance

    # Calculate median distance
    dist_values = [d[1] for d in dists]
    if dist_values:
        median_dist = median(dist_values)
    else:
        median_dist = 0

    # Set threshold (e.g., 3x median)
    threshold = 3 * median_dist
    outlier_indices = set(idx for idx, d in dists if d > threshold)

    # Rebuild clean CSV
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i, row in enumerate(rows):
            if i in outlier_indices:
                continue
            if all(x.strip() != '' for x in row[1:5]):
                writer.writerow(row)

    # Save metadata
    meta = {
        'obj1_pivot': pivot1,
        'obj2_pivot': pivot2
    }
    with open(json_path, 'w') as f:
        json.dump(meta, f, indent=4)

    print("Done. Data cleaned and saved.")


track_objects(
        video_path=R"C:\Users\avsha\Documents\GitHub\Ants\container\Videos\2024-09-18 (2 penduli & 2 slits)\S6130003.MP4",
    output_directory_path=R"C:\Users\avsha\Documents\GitHub\Ants\container\Analyzed Data\18.9.24\video 03pys",
    frame_skipping=10
)


Done. Data cleaned and saved.
